In [24]:
import tensorflow as tf
from tensorflow import keras
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import sys
sys.path.insert(1,'c:/Users/h2pro/Programming')
import StockPull
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay


#load data
company = 'tsla'
start = dt.datetime(2012,1,1)
end = dt.datetime(2022,1,1)

data = StockPull.analysisout(company)
df = data

In [29]:
# prepare the data for training
train_data = data[['Price', 'Volume']].values
train_dates = pd.to_datetime(data['Date'])
scaler = StandardScaler()
scaler = scaler.fit(train_data)
train_data_scaled = scaler.transform(train_data)


In [35]:
#Variables for training
cols = list(df)[1:3]
#New dataframe with only training data - 5 columns
df_for_training = df[cols].astype(float)
# print(train_data)
print(df_for_training)

           Price       Volume
0      21.008667   52027500.0
1      21.927334  119070000.0
2      22.735332  130998000.0
3      22.601999   93322500.0
4      22.525999   86998500.0
...          ...          ...
1253  194.699997   80046200.0
1254  194.860001   73533400.0
1255  182.449997   93122700.0
1256  179.820007   92150800.0
1257  174.039993   84052700.0

[1258 rows x 2 columns]


In [ ]:
# set up the training and test data
n_future = 1
n_past = 60

train_x = []
train_y = []

for i in range(n_past, len(train_data_scaled) - n_future +1):
    train_x.append(train_data_scaled[i - n_past:i, 0:train_data.shape[1]])
    train_y.append(train_data_scaled[i + n_future - 1:i + n_future, 0])

train_x, train_y = np.array(train_x), np.array(train_y)


In [ ]:
# specify the test split (e.g. 0.2 for 20% of the data to be used for testing)
test_split = 0.2

# split the data into training and test sets
n_test = int(test_split * len(train_x))
test_x, test_y = train_x[:n_test], train_y[:n_test]
train_x, train_y = train_x[n_test:], train_y[n_test:]

# print the shapes of the training and test sets
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)
print('test_x shape:', test_x.shape)
print('test_y shape:', test_y.shape)


In [ ]:
# build the model
model = keras.Sequential()

model.add(keras.layers.LSTM(64, activation='relu', input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=True))
model.add(keras.layers.LSTM(32, activation='relu', return_sequences=False))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(train_y.shape[1]))

# compile the model
model.compile(optimizer='adam', loss='mse')
model.summary()

# train the model
history = model.fit(train_x, train_y, epochs=30, batch_size=32)

In [ ]:
# evaluate the model
loss = model.evaluate(train_x, train_y)
print('Test Loss:', loss)

In [ ]:
# make predictions on the test set
yhat = model.predict(test_x)

# add an extra dimension to the yhat array
forecast_copies = np.repeat(yhat, train_data.shape[1], axis=-1)

# invert scaling for forecast
inv_yhat = scaler.inverse_transform(forecast_copies)
inv_yhat = inv_yhat[:,0]

In [ ]:
# add an extra dimension to the inv_y array
test_y_copies = np.repeat(test_y, train_data.shape[1], axis=-1)

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))

# inv_y = np.concatenate((test_y, test_x[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(test_y_copies)
inv_y = inv_y[:,0]


In [ ]:
# plot the predicted and actual prices
plt.plot(inv_y, label='Actual Price')
plt.plot(inv_yhat, label='Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title(company)
plt.legend()
plt.show()